# AMs with interactions under L0

$$
\begin{align}
    \min_{\beta_i, \theta_{ij}} & \frac{1}{2N}\left\lVert y - \left[\sum_{i}^{p} B_i \beta_i + \sum_{ij}^{p \choose 2} B_{ij} \theta_{ij} \right]\right\rVert_2^2
    + \lambda_1 \left[\sum_{i}^{p} \beta_i^T S_i \beta_i + \sum_{ij}^{p \choose 2} \theta_{ij}^T S_{ij} \theta_{ij} \right] +  \lambda_2 \left[\sum_{i}^{p} \mathbf{1}[\beta_i \neq 0] + \alpha \sum_{ij}^{p \choose 2} \mathbf{1}[\theta_{ij} \neq 0] \right] \\
\end{align}
$$

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
print(sys.version, sys.platform, sys.executable) #Displays what environment you are actually using.

In [ ]:
from __future__ import division
%matplotlib inline
from copy import deepcopy

import dill
import gc
from ipywidgets import *
import numpy as np
import pandas as pd

sys.path.insert(0, os.path.abspath(os.getcwd()).split('SparseAMsWithInteractions')[0])
from SparseAMsWithInteractions.src import data_utils
from SparseAMsWithInteractions.src import utils
from SparseAMsWithInteractions.src.AMsWithInteractionsL0 import models

# Import Processed Data

In [ ]:
load_directory='/home/shibal/Census-Data'
save_directory = os.path.join(os.path.abspath(os.getcwd()).split('src')[0], "results") 

df_X, df_y, _ = data_utils.load_data(load_directory=load_directory,
                                  filename='pdb2019trv3_us.csv',
                                  remove_margin_of_error_variables=True)
seed = 8
np.random.seed(seed)
X, Y, Xval, Yval, Xtest, Ytest, _, y_scaler = data_utils.process_data(
    df_X,
    df_y,
    val_ratio=0.1, 
    test_ratio=0.1,
    seed=seed,
    standardize_response=False)

### Initialize parameters

In [ ]:
convergence_tolerance = 1e-4
column_names = df_X.columns
r = 1.0
logging = True
max_interaction_support=400 # cuts off the L0 regularization path when the number of interactions reach 400.
version = 3.0
path = os.path.join(save_directory, 'AMsWithInteractionsL0', 'v{}'.format(version), 'r{}'.format(r))
os.makedirs(path, exist_ok=True)

In [ ]:
with open(path+'/Parameters.txt', "w") as f:
    f.write('Logging: {}, tol: {:.6f}, Train: {}, Validation: {}, Test: {}\n'.format(
        logging, convergence_tolerance, X.shape[0], Xval.shape[0], Xtest.shape[0])) 

In [ ]:
for c in column_names:
    print(c)

### How to run the model

Running the model over a 2 dimensional grid of hyperparameters (smoothing: $\lambda_1$, L0: $\lambda_2$) on ~300 covariates and ~45000 interaction effects requires large memory/time resources. 

For reproducibility, we provide a previous run of the model that saves the union of interaction effects that appeared in the model as we tune over the 2D grid. This leads to ~900 subset of interaction effects. 

We rerun the model on a 2D grid with ~300 covariates and ~900 interaction effects

In [ ]:
grid_search = 'full' # 'full', 'reduced'
load_model = False

In [ ]:
if grid_search=='full':
    generate_all_pairs = True
elif grid_search=='reduced':
    generate_all_pairs = False
    if load_model:
        folder = 'AMsWithInteractionsL0/v1.0/r1.0'
        with open(os.path.join(save_directory, folder, 'model.pkl'), 'rb') as input:
            ami_loaded = dill.load(input)
        # active_set = ami_loaded.active_set_union
        interaction_terms = ami_loaded.interaction_terms_union
        # active_set = np.sort(np.union1d(active_set, np.unique(interaction_terms)))
        # print("Number of main effects to consider:", len(active_set)) 
        print("Number of interaction effects to consider:", len(interaction_terms))
    else:
        # define specific interaction terms to consider
        interaction_terms = np.array([[0,1],
                                      [0,2]]) 

In [ ]:
Ki = 10
Kij = 5
eval_criteria = 'mse'
p = X.shape[1]
X = X[:, :p]
Xval = Xval[:, :p]
Xtest = Xtest[:, :p]
N, _ = X.shape
Xmin = np.min(np.vstack([X, Xval, Xtest]), axis=0)
Xmax = np.max(np.vstack([X, Xval, Xtest]), axis=0)
lams_sm_start = -3
lams_sm_stop = -6
lams_L0_start = 1
lams_L0_stop = -4
lams_sm=np.logspace(start=lams_sm_start, stop=lams_sm_stop, num=20, base=10.0)
lams_L0=np.logspace(start=lams_L0_start, stop=lams_L0_stop, num=100, base=10.0)

In [ ]:
am = models.AMI(lams_sm=lams_sm,
                lams_L0=lams_L0,
                convergence_tolerance=convergence_tolerance,
                eval_criteria=eval_criteria,
                path=path,
                max_interaction_support=max_interaction_support,
                terminate_val_L0path=False
                )
am.load_data(X, Y, y_scaler, column_names, Xmin, Xmax)

from scipy.special import comb

if generate_all_pairs:
    am.generate_interaction_terms(generate_all_pairs=True)
else:    
    am.interaction_terms = interaction_terms
    am.generate_all_pairs = False
    am.I = (int)(comb(am.p, 2, exact=False))
    am.Imax = len(interaction_terms)

am.generate_splines_and_quadratic_penalties(Ki=Ki, Kij=Kij)
am.fitCV(Xval, Yval)
am.evaluate_and_save(Xtest, Ytest)
am.Btrain = None
am.BtrainT_Btrain = None
am.Btrain_interaction = None
am.Btrain_interactionT_Btrain_interaction = None
with open(os.path.join(path, 'model.pkl'), 'wb') as output:
    dill.dump(am, output)

Second pass over the hyperparameter grid is much faster as it considers only the reduced set of promising interaction effects that appeared in the solution over the grid. Due to nonconvexity of $\ell_0$-norm, the solution quality improves with this second run!

In [ ]:
am_new = models.AMI(lams_sm=lams_sm,
                    lams_L0=lams_L0,
                    convergence_tolerance=convergence_tolerance,
                    eval_criteria=eval_criteria,
                    path=path,
                    max_interaction_support=max_interaction_support,
                    terminate_val_L0path=False
                    )
am_new.load_data(X, Y, y_scaler, column_names, Xmin, Xmax)

from scipy.special import comb
am_new.interaction_terms = interaction_terms
am_new.generate_all_pairs = False
am_new.I = (int)(comb(p, 2, exact=False))
am_new.Imax = len(interaction_terms)

am_new.generate_splines_and_quadratic_penalties(Ki=Ki, Kij=Kij)
am_new.fitCV(Xval, Yval)
am_new.evaluate_and_save(Xtest, Ytest)
am_new.Btrain = None
am_new.BtrainT_Btrain = None
am_new.Btrain_interaction = None
am_new.Btrain_interactionT_Btrain_interaction = None
with open(os.path.join(path, 'model_final.pkl'), 'wb') as output:
    dill.dump(am_new, output)